In [11]:
import os
import torch

REPO_URL = "https://github.com/thenileshmishra/AS-RoPE.git"
REPO_DIR = "/content/AS-RoPE"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
%cd {REPO_DIR}
!git pull

%pip install -q -r requirements.txt
print('cuda_available=', torch.cuda.is_available())
if torch.cuda.is_available():
    print('gpu=', torch.cuda.get_device_name(0))

/content/AS-RoPE
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.47 KiB | 1.47 MiB/s, done.
From https://github.com/thenileshmishra/AS-RoPE
   beef4a2..ee28b34  main       -> origin/main
Updating beef4a2..ee28b34
Fast-forward
 eval_perplexity.py |  1 +
 train.py           | 23 ++++++++++++++++++++++-
 2 files changed, 23 insertions(+), 1 deletion(-)
cuda_available= True
gpu= Tesla T4


In [ ]:
!python train.py \
  --seed 42 \
  --steps 2000 \
  --eval_interval 100 \
  --save_interval 2000 \
  --batch_size 8 \
  --block_size 1024 \
  --max_seq_len 8192 \
  --max_tokens 120000 \
  --checkpoint_path checkpoint_rope.pt \
  --metrics_path logs/rope_metrics.csv

Training on cuda
Parameters: 17.60M
use_as_rope=False
step    1 | train_loss 220.6299 | eval_loss 187.2761 | eval_ppl 2152716459906428978891779061430311182221223380367605791817579815617351352103993344.0000 | 0.9s
step  100 | train_loss 21.8992 | eval_loss 22.2225 | eval_ppl 4478411209.2792 | 50.3s
step  200 | train_loss 14.0313 | eval_loss 14.6057 | eval_ppl 2203707.7827 | 106.1s
step  300 | train_loss 10.1858 | eval_loss 9.8381 | eval_ppl 18735.0067 | 163.9s
step  400 | train_loss 7.7971 | eval_loss 7.9087 | eval_ppl 2720.8588 | 222.2s
step  500 | train_loss 6.8812 | eval_loss 6.7897 | eval_ppl 888.6601 | 280.5s
step  600 | train_loss 6.2802 | eval_loss 5.8864 | eval_ppl 360.0895 | 338.9s
step  700 | train_loss 5.3517 | eval_loss 5.6298 | eval_ppl 278.6057 | 397.2s


In [ ]:
!python train.py \
  --seed 42 \
  --steps 2000 \
  --eval_interval 100 \
  --save_interval 2000 \
  --batch_size 8 \
  --block_size 1024 \
  --max_seq_len 8192 \
  --max_tokens 120000 \
  --use_as_rope \
  --checkpoint_path checkpoint_as_rope.pt \
  --metrics_path logs/as_rope_metrics.csv

context=512 ppl=181.214614
context=1024 ppl=195.160161
context=2048 ppl=233.308794
context=4096 ppl=294.470651
degradation_beyond_1024=True


In [ ]:
!python eval_perplexity.py \
  --checkpoint_path checkpoint_rope.pt \
  --context_lengths 1024,2048,4096,8192 \
  --max_eval_tokens 32768

In [ ]:
!python eval_perplexity.py \
  --checkpoint_path checkpoint_as_rope.pt \
  --context_lengths 1024,2048,4096,8192 \
  --max_eval_tokens 32768

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

rope = pd.read_csv('logs/rope_metrics.csv')
as_rope = pd.read_csv('logs/as_rope_metrics.csv')

plt.figure(figsize=(8,5))
plt.plot(rope['step'], rope['eval_ppl'], label='RoPE baseline', linewidth=2)
plt.plot(as_rope['step'], as_rope['eval_ppl'], label='AS-RoPE', linewidth=2)
plt.xlabel('Step')
plt.ylabel('Perplexity')
plt.title('Perplexity vs Step')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()